https://superlinked.com/vector-db-comparison

### 🔹 Các component bên trong một **collection**:

1. **Points**

   * Là các bản ghi dữ liệu bạn thêm vào collection.
   * Mỗi point có:

     * `id`: định danh duy nhất.
     * `vector`: embedding (một hoặc nhiều vector tùy cấu hình).
     * `payload`: metadata (JSON key-value, để lọc hoặc phân loại).

2. **Vectors**

   * Thành phần cốt lõi để thực hiện similarity search.
   * Mỗi collection có thể chứa **nhiều vector space** (multi-vector), ví dụ: `"title_vector"`, `"body_vector"`.
   * Các vector có thể được gắn index để tối ưu tốc độ tìm kiếm.

3. **Payloads (Metadata)**

   * Dữ liệu phi cấu trúc dạng JSON đi kèm với mỗi point.
   * Ví dụ: `{ "category": "news", "author": "Alice" }`.
   * Dùng để lọc dữ liệu trong lúc search (`filtering`).

4. **Indexes**

   * Qdrant dùng index để tăng tốc truy vấn:

     * **Vector Index**: HNSW (Hierarchical Navigable Small World graph) cho tìm kiếm gần đúng (ANN).
     * **Payload Index**: dùng cho filter trên metadata (như B-Tree / Inverted Index).

5. **Filters**

   * Cho phép giới hạn kết quả dựa trên metadata (payload).
   * Ví dụ: tìm vector gần nhất nhưng chỉ trong `"category" = "sports"`.

6. **Scoring / Distance Functions**

   * Được định nghĩa khi tạo collection.
   * Các loại: Cosine similarity, Dot product, Euclidean (L2).

7. **Shards & Replicas**

   * **Shards**: collection có thể được chia nhỏ để phân tán dữ liệu (tăng hiệu năng).
   * **Replicas**: bản sao dữ liệu để tăng khả năng chịu lỗi (HA – High Availability).

8. **Snapshots & Backups**

   * Collection có thể tạo snapshot (sao lưu toàn bộ points, vectors, payloads) để backup hoặc migrate.

Cách chạy Qdran với Docker:

```docker run -p 6333:6333 -p 6334:6334 \
    -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
    qdrant/qdrant
```

In [1]:
import datasets

data = datasets.load_dataset("kiethuynhanh/vnlegal-dataset", split="train")
df = data.to_pandas()
df.head()

/Users/huynhanhkiet/Desktop/Fine-tune-LegalVN/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,field,question,context,answer,created_at
0,thue-phi-le-phi,"Thời điểm thực hiện khấu trừ, xác định số thuế...",,Căn cứ Điều 5 Nghị định 117/2025/NĐ-CP quy địn...,06/07/2025
1,thue-phi-le-phi,Chứng từ thanh toán không dùng tiền mặt được q...,,Căn cứ tại Điều 26 Nghị định 181/2025/NĐ-CP qu...,06/07/2025
2,thue-phi-le-phi,"Trách nhiệm của người bán hàng hóa, cung cấp d...",,Tại Điều 21 Nghị định 123/2020/NĐ-CP có quy đị...,06/07/2025
3,thue-phi-le-phi,"Căn cứ tính thuế xuất khẩu, thuế nhập khẩu đối...",,"Theo quy định Điều 5 Luật Thuế xuất khẩu, thuế...",06/07/2025
4,thue-phi-le-phi,TOÀN VĂN Công văn 1735 CT CS 2025 giới thiệu n...,,"Ngày 13 tháng 6 năm 2025, Cục Thuế đã ban hành...",06/07/2025


In [2]:
df_with_context = df[df['context'].str.len() > 0]
df_without_context = df[df['context'].str.len() == 0]

In [3]:
df_with_context.head()

,field,question,context,answer,created_at
5,thue-phi-le-phi,Cá nhân được hoàn thuế TNCN trong trường hợp nào?,Điều 8. Quản lý thuế và hoàn thuế\n1. Việc đăn...,Tại Điều 8 Luật Thuế thu nhập cá nhân 2007 có ...,06/07/2025
6,thue-phi-le-phi,Ký hiệu hóa đơn điện tử có mấy ký tự?,"Điều 5. Ký hiệu mẫu số, ký hiệu hóa đơn, tên l...",Tại Điều 5 Thông tư 32/2025/TT-BTC có quy định...,06/07/2025
9,thue-phi-le-phi,Thời điểm tính thuế xuất nhập khẩu là thời điể...,"Trị giá tính thuế, thời điểm tính thuế\n1. Trị...","Tại Điều 8 Luật Thuế xuất khẩu, thuế nhập khẩu...",06/07/2025
11,thue-phi-le-phi,Thời điểm xác định thuế giá trị gia tăng đối v...,Điều 8. Thời điểm xác định thuế giá trị gia tă...,Căn cứ theo điểm a khoản 1 Điều 8 Luật Thuế gi...,06/07/2025
12,thue-phi-le-phi,Lập hóa đơn điều chỉnh đối với chiết khấu thươ...,"Điều 19. Thay thế, điều chỉnh hóa đơn điện tử\...",Căn cứ theo Điều 19 Nghị định 123/2020/NĐ-CP s...,06/07/2025


In [4]:
from sentence_transformers import SentenceTransformer
from fastembed import SparseTextEmbedding 

dense_embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
sparse_embedding_model = SparseTextEmbedding("Qdrant/bm25")

In [5]:
from qdrant_client import QdrantClient
from qdrant_client.models import (
    models,
    Distance, 
    VectorParams, 
    PointStruct,
    CreateCollection,
    FieldCondition,
    MatchValue,
    Filter, Datatype, HnswConfigDiff, SparseVectorParams,
)
import uuid
import tqdm
from datetime import datetime
from qdrant_client.models import PointStruct

In [6]:
client = QdrantClient(host="localhost", port=6333, timeout=600.0)

In [7]:
collection_name="thue-phi-le-phi_all-MiniLM-L6-v2"

In [8]:
# Check if collection already exists
try:
    collection_info = client.get_collection(collection_name)
    print(f"Collection '{collection_name}' already exists. Skipping creation.")
except Exception:
    print(f"Creating new collection '{collection_name}'...")
    client.create_collection(
            collection_name=collection_name,
            vectors_config={
                # Vector dense với HNSW + cosine
                "dense": VectorParams(
                size=384,
                distance=Distance.COSINE,
                datatype=Datatype.FLOAT16,
                on_disk=True,
                # cấu hình HNSW: điều chỉnh theo dữ liệu thực tế
                hnsw_config=HnswConfigDiff(
                    m=16,                 # số cạnh / node
                    ef_construct=200,     # chất lượng khi build
                    ),
                ),
            },
            # Khai báo không gian vector sparse cho BM25
            sparse_vectors_config={
                "bm25": SparseVectorParams(modifier=models.Modifier.IDF)
            }
    )

Collection 'thue-phi-le-phi_all-MiniLM-L6-v2' already exists. Skipping creation.


In [9]:
dense_embedding = dense_embedding_model.encode(df_with_context['context'].values[0]) # <--- Dense vector
bm25_embedding = sparse_embedding_model.embed(df_with_context['context'].values[0]) # <--- Sparse vector

In [10]:
# Check if collection already has data
collection_info = client.get_collection(collection_name)
if collection_info.points_count > 0:
    print(f"Collection already has {collection_info.points_count} points. Skipping data insertion.")
else:
    print("Inserting data into collection...")
    for context in tqdm.tqdm(df_with_context['context'].values):
        
        idx = str(uuid.uuid4())
        create_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S") #'2025-08-16 16:51:35'
        dense_embedding = dense_embedding_model.encode(context) # <--- Dense vector
        bm25_embedding = sparse_embedding_model.embed(context) # <--- Sparse vector
        
        # Convert sparse embedding to proper format
        bm25_vector = list(bm25_embedding)[0]
        
        operation_info = client.upsert(
            collection_name=collection_name,
            wait=True,
            points=[
                PointStruct(id=idx, 
                            vector={
                                "dense": dense_embedding, 
                                "bm25": bm25_vector.as_object()
                            },
                            payload={
                                "create_at": create_at,
                                "raw_context": context,
                            }
                        ),
                ],
        )

Collection already has 614 points. Skipping data insertion.


In [11]:
collection_info

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=614, points_count=614, segments_count=4, config=CollectionConfig(params=CollectionParams(vectors={'dense': VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=HnswConfigDiff(m=16, ef_construct=200, full_scan_threshold=None, max_indexing_threads=None, on_disk=None, payload_m=None), quantization_config=None, on_disk=True, datatype=<Datatype.FLOAT16: 'float16'>, multivector_config=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors={'bm25': SparseVectorParams(index=None, modifier=<Modifier.IDF: 'idf'>)}), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=10

In [12]:
# test_context = df_with_context['context'].values[0]
test_context = df_with_context['question'].values[0]
dense_vector_query = dense_embedding_model.encode(test_context)
bm25_vector_query = sparse_embedding_model.embed(test_context)

# Convert query sparse vector to proper format
bm25_query_vector = list(bm25_vector_query)[0]

In [13]:
search_result = client.query_points(
    collection_name=collection_name,
    query=models.FusionQuery(
            fusion=models.Fusion.RRF  # we are using reciprocal rank fusion here
        ),
    prefetch=[
            models.Prefetch(
                query=dense_vector_query,
                using="dense",
                limit=10
            ),
            models.Prefetch(
                query=bm25_query_vector.as_object(),
                using="bm25",
                limit=10
            ),
        ],
    with_payload=True,
    with_vectors=True,
    limit=20
).points

In [14]:
df_with_context['context'].values[0]

'Điều 8. Quản lý thuế và hoàn thuế\n1. Việc đăng ký thuế, kê khai, khấu trừ thuế, nộp thuế, quyết toán thuế, hoàn thuế, xử lý vi phạm pháp luật về thuế và các biện pháp quản lý thuế được thực hiện theo quy định của pháp luật về quản lý thuế.\n2. Cá nhân được hoàn thuế trong các trường hợp sau đây:\na) Số tiền thuế đã nộp lớn hơn số thuế phải nộp;\nb) Cá nhân đã nộp thuế nhưng có thu nhập tính thuế chưa đến mức phải nộp thuế;\nc) Các trường hợp khác theo quyết định của cơ quan nhà nước có thẩm quyền.'

In [15]:
import pandas as pd

print(f"\nSearch results: {len(search_result)} points found")
print(f"\nQuery: {test_context}")
df_search_result = pd.DataFrame(
    {'id': [point.id for point in search_result],
    'context': [point.payload.get('raw_context', 'N/A') for point in search_result],
    'score': [point.score if hasattr(point, 'score') else 'N/A' for point in search_result]
    })
df_search_result


Search results: 20 points found

Query: Cá nhân được hoàn thuế TNCN trong trường hợp nào?


,id,context,score
0,af5d66b7-c757-4484-aeae-75119af7f56c,ĐỐI TƯỢNG PHẢI QUYẾT TOÁN THUẾ\n1. Đối với cá ...,0.500000
1,d8eb223d-54a4-4573-a5b9-63e73b3687f0,Khấu trừ thuế và chứng từ khấu trừ thuế\n1. Kh...,0.500000
2,5743dc8b-2db6-43ea-9727-95e35da15e34,Khấu trừ thuế và chứng từ khấu trừ thuế\n1. Kh...,0.333333
3,16486976-d09f-4166-b1a1-8ad7863da25b,I. ĐỐI TƯỢNG PHẢI QUYẾT TOÁN THUẾ\n1. Đối với ...,0.333333
4,219f2445-d496-468d-a73f-756c246e5927,ĐỐI TƯỢNG PHẢI QUYẾT TOÁN THUẾ\n....\n2. Đối v...,0.250000
5,d1e33c85-1121-41ad-bf9c-78934091b9cf,Các trường hợp không thực hiện khấu trừ thuế\n...,0.250000
6,bff51378-e1e1-4c8d-9d77-d3afb9cab5c7,Điều 5. Đối tượng không chịu thuế\n[...]\n26. ...,0.200000
7,3fd190bf-0a48-4fe8-b501-052ba8f0557e,Điều 3. Miễn lệ phí môn bài\nCác trường hợp đư...,0.200000
8,0943dad9-7449-4023-ac36-874267d32d62,Điều 13. Áp dụng hóa đơn điện tử khi bán hàng ...,0.166667
9,c87dd96d-8b74-48e9-9420-57892da5a681,| - Tên nền tảng thương mại điện tử (cột 02): ...,0.166667


In [27]:
initial_hits = []
for i, hit in enumerate(search_result):
    initial_hits.append(hit.payload["raw_context"])

In [16]:
from fastembed.rerank.cross_encoder import TextCrossEncoder

model_reranker = TextCrossEncoder(model_name='jinaai/jina-reranker-v2-base-multilingual')

Fetching 5 files: 100%|██████████| 5/5 [00:12<00:00,  2.57s/it]


In [31]:
new_scores = list(
    model_reranker.rerank(test_context, initial_hits)
)  # returns scores between query and each document

ranking = list(enumerate(new_scores))
ranking.sort(
    key=lambda x: x[1], reverse=True
)  # sorting them in order of relevance defined by reranker

In [45]:
df_rerank_result = pd.DataFrame(
    {
        "Rank": range(1, len(ranking) + 1),
        "Document": [initial_hits[i] for i, _ in ranking],
        "Score": [score for _, score in ranking],
    }
)

df_rerank_result

,Rank,Document,Score
0,1,I. ĐỐI TƯỢNG PHẢI QUYẾT TOÁN THUẾ\n1. Đối với ...,-0.086371
1,2,ĐỐI TƯỢNG PHẢI QUYẾT TOÁN THUẾ\n....\n2. Đối v...,-0.282917
2,3,ĐỐI TƯỢNG PHẢI QUYẾT TOÁN THUẾ\n1. Đối với cá ...,-0.439669
3,4,| - Tên nền tảng thương mại điện tử (cột 02): ...,-0.527150
4,5,- Căn cứ Điều 25 Thông tư số 111/2013/TT-BTC n...,-0.594118
5,6,Phương pháp tính thuế đối với một số trường hợ...,-0.649784
6,7,Điều 3. Miễn lệ phí môn bài\nCác trường hợp đư...,-0.781844
7,8,Điều 14. Khấu trừ thuế giá trị gia tăng đầu và...,-1.420984
8,9,Điều 5. Đối tượng không chịu thuế\n[...]\n26. ...,-1.488117
9,10,Điều 25. Thuế đối với thu nhập từ kinh doanh\n...,-1.539383
